Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). On JupyterLab, you may want to hit the "Validate" button as well.

Caution: do not mess with the notebook's metadata; do not change a pre-existing cell's type; do not copy pre-existing cells (add new ones with the + button instead). This will break autograding; you will get a 0; you are warned.

<table style="width: 100%; border: none;" cellspacing="0" cellpadding="0" border="0">
  <tr>
    <td><img src="https://www.planetegrandesecoles.com/wp-content/uploads/2021/07/Identite%CC%81-visuelle-Plane%CC%80te-BAC-8-600x398.png" style="float: left; width: 100%" />
</td>
    <td><a style="font-size: 3em; text-align: center; vertical-align: middle;" href="https://moodle.polytechnique.fr/course/view.php?id=19260">[CSC2S004EP - 2024] - Introduction to Machine Learning</a>
</td>
  </tr>
</table>

<a style="font-size: 3em;">Lab Session 4: Overfitting and Regularization</a>

Jérémie DECOCK - Adrien EHRHARDT - Jesse READ

# Objectives

Earlier, we have used a **parametric model** to solve **regression problems** (lab 02: linear and polynomial regression, closed form and gradient descent, by hand and with `sklearn`!). And we used logistic regression, a **parametric model** on **classification tasks** and k-NN, a **non-parametric model** on classification **and** regression problems (in lab 03).

Today we will continue the exploration of these methods, with a specific emphasis on understanding and correcting issues with overfitting, with regularization, and the bias-variance tradeoff. 

**Note**: as in the previous labs, there are some differences in notation with the lecture slides. For instance, parameters are noted $w$ (machine learning community) in lectures but they are noted $\theta$ here (statistics community).

# Imports

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing
from sklearn.utils import shuffle

# Synthetic Data

Toy data is not just for play. If we define and control the ground truth, we can check (and, thus, further develop) many theoretical properties of methods, in a way which is not possible with real-world data.

In the following cell, provide the function `generate` which should return two 1d arrays (`x` and `y`) each of length `n` such that `x[i]` is a number generated uniformly between 0 and 10, and `y[i]` is:
$$
    y_i \sim \mathcal{N}(\boldsymbol{\theta}_\star^T\mathbf{x}_i, \sigma^2)
$$
for $\boldsymbol\theta_\star = [-0.6, 2]$ and $\sigma=0.7$; where $\mathbf{x}_i = [1, x_i]$.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

def generate(n=20, x_min=0, x_max=10):
    # YOUR CODE HERE
    raise NotImplementedError()
    return x, y

In [ ]:
x, y = generate()
assert x.shape == (20,)
assert y.shape == (20,)
assert x.max() <= 10
assert x.min() >= 0

# Warmup: Ordinary Least Squares

Implement ordinary least squares in the following function. It should return the vector of coefficients $\mathbf{\theta}$. You may ignore the `alpha` parameter (corresponding to $\lambda$ in the lectures -- since the `lambda` keyword is already taken in Python) for now; we will use that later. 

In [ ]:
def least_squares(X: np.array, y: np.array, alpha: float = 0.0) -> np.array:
    """
    Perform linear regression via least squares, return coefficient

    :param numpy.array X: design matrix (n-sample as rows, p features as columns, may contain column of 1 to get an intercept term)
    :param numpy.array y: response vector (n elements)
    :param numpy.array alpha: regularization parameter (aka lambda; between 0 and inf - see Exercise 7)
    :return: linear regression coefficient found via ols
    :rtype: numpy.array
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return theta

We build $\mathbf{x}_i = [1, x_i]$ as follows:

In [ ]:
X = np.vstack([np.ones_like(x), x]).T

We compute $\hat{\boldsymbol{\theta}}$ as follows:

In [ ]:
theta_hat = least_squares(X, y)
assert theta_hat.shape == (2,)

In the following block, compute predictions on the training data and calculate the residual sum of squares (RSS) error as $\sum_{i=1}^n (\hat y_i - y_i)^2$, and store the result in the variable `E_RSS`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
E_RSS

How does this compare to the error we would get if we had available the true model (the same one used to generate the data)?

Repeat the RSS calculation but for the predictions obtained from the true model $f_\star$; and this time store the result in the variable `E_RSS_val`:

In [ ]:
theta_star = np.array([-0.6, 2])  # this is theta_star used to generate the data, see 3.
# YOUR CODE HERE
raise NotImplementedError()
E_RSS_val

Those number should be pretty close, most of the time (for most data sets that you could generate), since (and it's worth taking a while to check/understand this) all assumptions for ordinary least squares, regarding the data and underlying model, have been met.

Let's have a look at what is going on:

In [ ]:
n = len(x)
X = np.vstack([np.ones_like(x),x]).T
theta_hat = least_squares(X, y)
yp = X @ theta_hat
yt = X @ theta_star
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x, y)
ax.plot(x, yp, label="$\\hat f$")
ax.plot(x, yt, label="$f_\\star$")
plt.legend()
plt.show()

# Unbiased Estimators 

In the following, check empirically if your implementation of least squares is an *unbiased estimator*.

*Hint:* the derivation for this involves an expectation ($\mathbb{E}$) operator, but empirically you can replace this with a sum over many samples. You might check the slides/notes if you need a reminder/more information about what makes a model biased or not. 

In [ ]:
print(theta_star)

# Of course, there are many ways to do this, it is the result (avg_theta) that counts... 
theta_hats = []
for _ in range(1000):
    # Generate dataset, estimate theta_hat, average theta_hats
    # theta_ = [...]
    # YOUR CODE HERE
    raise NotImplementedError()
    theta_hats.append(theta_hat_)
avg_theta = np.mean(theta_hats, axis=0)  # acts as expectation over dataset D
avg_theta

So far so good. However, things are not always so easy. Assumptions are often violated. In the following we look at 'pathological cases' where least squares is not going to sufficiently solve our learning problem. 

# Pathological Cases (Afflicting OLS)

Observe the effect with the following datasets. Identify the issue with each one (why does least squares fail), among:

In [ ]:
possible_answers = {
    "A": "More features than samples",
    "B": "y cannot be approximated by a linear combination of x",
    "C": "Collinearity",
    "D": "There should be a column of 1s"
}

In [ ]:
X = np.array([[1, 2, 3, 4, 5], [2, 4, 6, 8, 10]]).T
y = np.array([1.8, 2.7, 3.4, 3.8, 3.9])

X

In [ ]:
# theta = least_squares(X, y)   # <- **TODO: UNCOMMENT**
# theta                         # <- **TODO: UNCOMMENT**

In [ ]:
# issue = "A" or "B" or ...  # <- **TODO: UNCOMMENT**
# YOUR CODE HERE
raise NotImplementedError()

And what about this dataset:

In [ ]:
X = np.array([[1, 3, 0, 2],
              [2, 3, 4, 4]])
y = np.array([1.8, 2.7])

X

In [ ]:
# theta = least_squares(X, y)   # <- **TODO: UNCOMMENT**
# theta                         # <- **TODO: UNCOMMENT**

In [ ]:
# issue = "A" or "B" or ...  # <- **TODO: UNCOMMENT**
# YOUR CODE HERE
raise NotImplementedError()

# Regularization with Ridge regression

Let's try a different dataset: 

In [ ]:
# Generate some data
np.random.seed(123)
x = np.random.rand(10) + 0.5
y = 1.5 * x**2 + np.random.randn(len(x)) * 0.07

# Build OLS
X = np.vstack([np.ones_like(x), x]).T
theta_hat = least_squares(X, y)
yp = X @ theta_hat

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x, y)
ax.plot(x, yp)
plt.ylim([min(y - 0.1), max(y + 0.1)])
plt.show()

Take a close look at the code for generating this data. Note how the dataset is generated from a *non-linear* function.

We apply basis expansion to fit a polynomial model to the data:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
Z = PolynomialFeatures(degree=8, include_bias=True).fit_transform(x.reshape(-1, 1))

# Instantiate and fit the model
poly_reg = sklearn.linear_model.LinearRegression(fit_intercept=False)  # bias is already in the polynomial expansion
poly_reg.fit(Z, y)
theta_hat_poly = poly_reg.coef_
print("Coefs:", theta_hat_poly)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x, y)
xxx = np.linspace(min(x) - 0.1, max(x) + 0.1, 100)
XXX = np.vstack([np.ones_like(xxx), xxx]).T
ax.plot(xxx, XXX @ theta_hat, label="$\\hat f$")
zzz = PolynomialFeatures(degree=8, include_bias=True).fit_transform(xxx.reshape(-1, 1))
plt.plot(xxx,poly_reg.predict(zzz),label="$\\hat f'$")
plt.ylim([min(y - 0.1), max(y + 0.1)])
plt.legend()
plt.show()

**Consider some of the issues here**, such as the number and magnitude of the coefficients, potential performance on the test data, in certain ranges of the $x$-domain.

A solution is to reduce the complexity of the model using a lower polynomial degree, e.g., with *ridge regularization* (a.k.a. *L2 regularization*) which applies a penalty on the values of $\theta_j$ to constrain them to be smaller. A parameter $\boldsymbol{\theta}$ with small magnitude usually makes the model simpler and brings better generalization performances. This L2 regularization is included in the least square method as follows:

$$
\boldsymbol{\theta}^*
\leftarrow \arg\min_{\boldsymbol{\theta}} E(\boldsymbol{\theta})
\quad \text{with} \quad
E(\boldsymbol{\theta})
= \underbrace{||\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta}||^2_2}_{\text{error term}} ~~ + \underbrace{\lambda ||\boldsymbol{\theta}||^2_2}_{\text{regularization}}$$

where $\lambda \in \mathbb{R}^+$ is the *regularization strength* coefficient:
- when $\lambda$ goes to infinity, the regularization term dominates the error term (MSE) and the coefficients $\boldsymbol{\theta}$ tend to zero;
- when $\lambda$ goes to 0, the regularization term looses the importance and eventually the regularization term is ignored;
- $\lambda$ is a *meta* or *hyper parameter*;
- the best $\lambda$ for a problem can be estimated empirically, we look at this soon!

On a sheet of paper or in the cell below (either with LaTeX or by embedding an image):
- Compute the analytic formulation of the gradient $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$
- Compute the analytic formulation of the optimal parameter $\boldsymbol{\theta^*}$

*Note*: this is very similar to [`lab_session_02`](https://adimajo.github.io/CSE204-2022-admin/lab_session_02.html).

Recall:

$$
(\boldsymbol{A} + \boldsymbol{B})^T = \boldsymbol{A}^T + \boldsymbol{B}^T
$$

$$
(\boldsymbol{A} \boldsymbol{B})^T = \boldsymbol{B}^T \boldsymbol{A}^T
$$

\begin{align}
\text{Associativity:} & \quad
(\boldsymbol{A} \boldsymbol{B}) \boldsymbol{C}
= \boldsymbol{A} (\boldsymbol{B} \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} \boldsymbol{C}
\\
\text{Non-commutativity:} & \quad
\boldsymbol{A} \boldsymbol{B}
\neq \boldsymbol{B} \boldsymbol{A}
\\
\text{Distributivity:} & \quad
\boldsymbol{A} (\boldsymbol{B} + \boldsymbol{C})
= \boldsymbol{A} \boldsymbol{B} + \boldsymbol{A} \boldsymbol{C} \\
& \quad
(\boldsymbol{B} + \boldsymbol{C}) \boldsymbol{A} 
= \boldsymbol{B} \boldsymbol{A} + \boldsymbol{C} \boldsymbol{A}
\end{align}

Differential identities for matrices:

\begin{align}
d(\boldsymbol{A}) & = 0  \quad \quad \quad \text{if } \boldsymbol{A} \text{ is not function of } \boldsymbol{X} \\
d(a\boldsymbol{X}) & = a d \boldsymbol{X}  \quad \quad \text{if } a \text{ is not function of } \boldsymbol{X} \\
d(\boldsymbol{X} +\boldsymbol{Y}) & = d \boldsymbol{X} + d \boldsymbol{Y} \\
d(\boldsymbol{XY}) & = (d \boldsymbol{X}) \boldsymbol{Y} + \boldsymbol{X} (d \boldsymbol{Y}) \\
d(\boldsymbol{X}^T) & = (d \boldsymbol{X})^T \\
\end{align}

YOUR ANSWER HERE

**Question**: Is it a convex optimization problem like *Ordinary Least Squares*? Why?

YOUR ANSWER HERE

Check the following Scikit Learn implementation of the Ridge Regression (more info here: https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression ). In particular: Change the `alpha` ($\lambda$) and `k`/degree parameters, and observe the results. 

In [ ]:
# Set hyper-parameters
alpha = 0  # regularization #TODO change to, e.g., 0.1, 1, etc.
k = 8  # degree polynomial  #TODO change to, e.g., 8, 4, 2, etc.
Z = PolynomialFeatures(degree=k, include_bias=True).fit_transform(x.reshape(-1, 1))

# Fit ridge regression
poly_ridge = sklearn.linear_model.Ridge(alpha=alpha, fit_intercept=False)
poly_ridge.fit(Z, y)

# Have a look at the coefficients
coefs = [poly_ridge.intercept_] + poly_ridge.coef_
print("Coefs:", coefs)

# Plot the result
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x, y)
xxx = np.linspace(min(x) - 0.1, max(x) + 0.1, 100)
zzz = PolynomialFeatures(degree=k, include_bias=True).fit_transform(xxx.reshape(-1, 1))
plt.plot(xxx,poly_ridge.predict(zzz),label="$\\hat f$ (deg. %d, $\\lambda=%3.2f$)" % (k, alpha), color='orange')
plt.ylim([min(y - 0.1), max(y + 0.1)])
plt.legend()
plt.show()

In the following, plot the ridge coefficients as a function of the regularization parameter.

Evaluate the following sequence of regularization strength: `alphas = np.logspace(-2, 5, 50)`.

In [ ]:
# Compute paths
alphas = np.logspace(-2, 5, 50)

coefs = []
for a in alphas:
    # Fit a `Ridge` object
    # coefs.append(...)  # TO UNCOMMENT: append the ridge coefficients to the `coefs` list
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Display results
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('Regularization')
plt.ylabel('Coefficients')
plt.title('Ridge coefficients as a function of the regularization')
plt.axis('tight')
plt.show()

**Return to the function `least_squares` that you implemented already above**; now update the function to implement the ridge regression in Python (*without* using Scikit-Learn). The ridge/regularization should 'take effect' *only when* `alpha > 0`. Observe your result in the following (you can experiment with `k` and `alpha` again):

In [ ]:
# Set the hyper-parameters
k = 2
alpha = 0.05

In [ ]:
# Test your function
Z = PolynomialFeatures(degree=k, include_bias=True).fit_transform(x.reshape(-1, 1))
theta_ridge = least_squares(Z, y, alpha=alpha)
print("Coefs:", theta_ridge)

# Plot the results
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(x, y)
xxx = np.linspace(min(x) - 0.1, max(x) + 0.1, 100)
zzz = PolynomialFeatures(degree=k, include_bias=True).fit_transform(xxx.reshape(-1, 1))
plt.plot(xxx,zzz @ theta_ridge, label="$\\hat f$ (deg. %d, $\\lambda=%3.2f$)" % (k,alpha), color='salmon')
plt.ylim([min(y - 0.1), max(y + 0.1)])
plt.legend()
plt.show()

# Ridge Logistic Regression

In the following we generate and plot a toy dataset for classification.

In [ ]:
def gen_2d_classification_samples(n: int = 20) -> np.array:
    """
    Generates 2-dimensional samples, which belong to either of 2 classes

    :param int n: number of samples to draw per class
    :returns: two arrays of X (shape (n,2)) and y (shape (n,)
    """
    y = ((np.random.rand(n) >= 0.5) * 1).astype(int)
    X = np.random.randn(n, 2) * 0.5 + (y.reshape(n, -1) == 1) * 1
    
    return X, y

In [ ]:
X,y = gen_2d_classification_samples(100)
x_min, x_max = np.array((X[:, 0], X[:, 1]))[0, :].min() - .5, np.array((X[:, 0], X[:, 1]))[0, :].max() + .5
y_min, y_max = np.array((X[:, 0], X[:, 1]))[1, :].min() - .5, np.array((X[:, 0], X[:, 1]))[1, :].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

We used gradient descent to fit a Logistic Regression, which finds a linear decision boundary between these two classes, and obtained the same results as `sklearn` in lab 03.

In [ ]:
# Recall the `LogisticRegression` class:
logistic_regression = sklearn.linear_model.LogisticRegression(C=1e9).fit(X, y)

In [ ]:
Z = logistic_regression.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 0]
plt.pcolormesh(xx, yy, (Z.reshape(xx.shape) > 0.5) * 1, cmap=plt.cm.Paired)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

Let's try to do polynomial logistic regression, with the same methodology: (1) fit a `PolynomialFeatures` object, that is, a polynomial expansion of x, from 1 = `x**0` to `x**degree`, (2) use that as an input to `LogisticRegression` (without an intercept term).

In [ ]:
fil = PolynomialFeatures(degree=7, include_bias=True)
Phi = fil.fit_transform(X)

logistic_regression = sklearn.linear_model.LogisticRegression(
    C=1e9, solver='liblinear', fit_intercept=False).fit(Phi, y)

In [ ]:
xxxx = np.c_[xx.ravel(), yy.ravel()]
phi_ = fil.transform(xxxx)
Z = logistic_regression.predict_proba(phi_)[:, 0]
plt.pcolormesh(xx, yy, (Z.reshape(xx.shape) > 0.5) * 1, cmap=plt.cm.Paired)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

You might observe a relatively poor fit, in the sense that it probably won't *generalize* well. Just like linear regression, let's try to regularize logistic regression with a ridge penalty ($\lambda ||\boldsymbol{\theta}||_2^2$).
On a sheet of paper (that you drag-and-drop in the cell below or in $\LaTeX$), compute the analytic formulation of the gradient $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$.

*Note*: this is similar to above **but** the error function is now the log-loss.

YOUR ANSWER HERE

We can use the `C` parameter in `sklearn`, inversely proportional to $\lambda$, to fit such a penalization. Play around with `C` in the following cell, than plot the resulting decision boundary in the subsequent cell.

In [ ]:
# Lower C means more regularization (higher alpha) - you may edit C:
logistic_regression = sklearn.linear_model.LogisticRegression(
    C=0.1, solver='liblinear', penalty='l2', fit_intercept=False).fit(Phi,y)

In [ ]:
xxxx = np.c_[xx.ravel(), yy.ravel()]
phi_ = fil.transform(xxxx)
Z = logistic_regression.predict_proba(phi_)[:, 0]
plt.pcolormesh(xx, yy, (Z.reshape(xx.shape) > 0.5) * 1, cmap=plt.cm.Paired)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.show()

# Cross Validation and Hyper-Parameter Tuning

What value of `C` should we use? Consider the potential values listed in `c_range` in the following code block. Make use of the `cross_val_score` function which will handle cross validation for you. You should do 10-fold cross validation, under the 'accuracy' scoring metric; use logistic regression as the `estimator`, and the `PolynomialFeatures` (degree 7) `Phi`, as produced recently (above). Consider the average accuracy across the 10 folds as the 'value' of each `C`-value; choose the one you prefer and put it in `c_selected`.

*Hint*: see [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

In [ ]:
# This is the kind of model you should test on: 
# sklearn.linear_model.LogisticRegression(C=c, solver='liblinear', penalty='l2')

# ... for the following values of c: 
c_range = [0.001, 0.01, 0.1, 1, 10, 50, 100, 200, 300, 400, 500, 1000]

# Make use of the following 
from sklearn.model_selection import cross_val_score

In [ ]:
# You should update the following variable with your choice
c_selected = c_range[0]

# YOUR CODE HERE
raise NotImplementedError()

# Weighted Least Squares (Bonus)

In [ ]:
def gen_1d_polynomial_regression_samples(n_samples: int = 15) -> pd.DataFrame:
    """
    Generate 1-dimensional regression samples (x, y)

    :param int n_samples: how many samples to return
    """
    x = np.random.uniform(low=0., high=1.5, size=n_samples)
    y = np.cos(2. * np.pi * x) + np.random.normal(scale=0.1, size=x.shape)
    df = pd.DataFrame(np.array([x, y]).T, columns=['x', 'y'])
    df = sklearn.utils.shuffle(df).reset_index(drop=True)
    return df

In [ ]:
def plot_regression_1d(X, y, theta=None, x_min=0, x_max=2):
    """
    Plot linear regression of X on y given theta
    """
    assert X.ndim == 2 and X.shape[1] == 2, X.shape
    assert y.ndim == 2 and y.shape[1] == 1, y.shape
    if theta is not None:
        assert theta.ndim == 2 and theta.shape == (2, 1), theta.shape
    
    fig, ax = plt.subplots()
    ax.scatter(X[:,1], y)

    if theta is not None:
        x = np.linspace(x_min, x_max, 50)
        y = theta[0] + theta[1] * x

        ax.plot(x, y, "--r")
    plt.show()

In [ ]:
def plot_1d_regression_samples(dataframe: pd.DataFrame, model=None):
    """
    Plot the data in dataframe, as wellas (optionnally) the predictions from a model

    :param pandas.DataFrame dataframe: dataframe containing 'x' and 'y'
    :param model: model to predict
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    df = dataframe.copy()  # make an alias
    
    ERROR_MSG1 = "The `dataframe` parameter should be a Pandas DataFrame having the following columns: ['x', 'y']"
    assert df.columns.values.tolist() == ['x', 'y'], ERROR_MSG1
    
    if model is not None:
        
        # Compute the model's prediction
        x_pred = np.linspace(df.x.min(), df.x.max(), 100).reshape(-1, 1)
        y_pred = model.predict(x_pred)
        df_pred = pd.DataFrame(np.array([x_pred.flatten(), y_pred.flatten()]).T, columns=['x', 'y'])
        df_pred.plot(x='x', y='y', style='r--', ax=ax)

    # Plot also the training points
    df.plot.scatter(x='x', y='y', ax=ax)
    delta_y = df.y.max() - df.y.min()
    plt.ylim((df.y.min() - 0.15 * delta_y,
              df.y.max() + 0.15 * delta_y))
    plt.show()

For some regression problems, it may be helpful to give different importance to examples in the *learning set* $\mathcal{D} = \{(\boldsymbol{x^{(i)}}, y^{(i)})\}_{1 \leq i \leq n}$ that is to say associate a weight $\omega^{(i)}$ to example $\boldsymbol{x}^{(i)}$ in order to prioritize some of them and ignore some others (e.g. outliers).
Introducing these weights in the method of Least Square, the regression problem becomes:

$$E(\boldsymbol{\theta}) = \sum_{i=1}^n \omega^{(i)} (y^{(i)} - \boldsymbol{x}^{(i)} \boldsymbol{\theta})^2.$$

In order to use the matrix notation, we put weights $\omega^{(i)}$ in the diagonal of the following matrix $\Omega$:

$$
\Omega =
\begin{pmatrix}
\omega^{(1)} & 0            & \cdots & 0 \\
0            & \omega^{(2)} & \cdots & 0 \\
\vdots       & \vdots       & \ddots & \vdots \\
0            & 0            & \cdots & \omega^{(n)} \\
\end{pmatrix}
$$

Then we can write:

$$E(\boldsymbol{\theta}) = (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta})^T \Omega (\boldsymbol{y} - \boldsymbol{X} \boldsymbol{\theta})$$

with:
$$
\boldsymbol{X} = \begin{pmatrix} 1 & x_1^{(1)} & \dots & x_p^{(1)} \\ \vdots & \vdots & \dots & \vdots \\ 1 & x_1^{(n)} & \dots & x_p^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{y} = \begin{pmatrix} y^{(1)} \\ \vdots \\ y^{(n)} \end{pmatrix}
\quad \quad
\boldsymbol{\theta} = \begin{pmatrix} \theta_0 \\ \vdots \\ \theta_p \end{pmatrix}
$$


On a sheet of paper (that you drag-and-drop in the following cell or with $\LaTeX$):
- Compute the analytic formulation of the gradient $\nabla_{\boldsymbol{\theta}} E(\boldsymbol{\theta})$
- Compute the analytic formulation of the optimal parameter $\boldsymbol{\theta^*}$

*Note*: this is again very similar to question 2.1.

YOUR ANSWER HERE

Is this problem a convex optimization problem like *Ordinary Least Squares*? Why?

YOUR ANSWER HERE

We have the following dataset and weights:

In [ ]:
X = np.array([[1, 1],
              [1, 2],
              [1, 3],
              [1, 4],
              [1, 5]])

y = np.array([1.8, 4.5, 3.4, 3.6, 4.2]).reshape([-1, 1])

Omega = np.diag([1, 2, 3, 2, 1])

Omega

In [ ]:
plt.scatter(X[:,1], y)
plt.show()

Complete the following Python implementation of the `weighted_least_squares()` procedure.
It should return the optimal parameter $\boldsymbol{\theta^*}$ using the method of Least Square for the matrix of weights $\Omega$ defined above. Here, we expect $\theta$ to be a Numpy array of shape `(1, 2)` (i.e. a vector of two elements).

*Numpy hints*:
- The transpose of a matrix `X` is obtained with `X.T`
- The inverse of a matrix `X` is obtained with `np.linalg.inv(X)`
- The product of two matrices `X` and `Y` is obtained with `np.dot(X, Y)` or `np.matmul(X, Y)` or `X @ Y`
- The dot product of a matrix `X` and a vector `y` is obtained with `np.dot(X, y)`

In [ ]:
def weighted_least_squares(X, Omega, y):
    # theta = ...  # TO UNCOMMENT AND COMPLETE
    # YOUR CODE HERE
    raise NotImplementedError()
    return theta

In [ ]:
theta = weighted_least_squares(X, Omega, y)
assert len(theta) == 2

Check graphically your model using the following code:

In [ ]:
plot_regression_1d(X, y, theta, x_min=0, x_max=6)

This is arguably relatively bad, since it seems the second point might be an outlier and "drags" the regression line.

Change the weights in $\Omega$ to ignore the second point $x = 2$ (give the same weight to all other points) then recompute $\theta$ using `weighted_least_squares()` and check the results on plots with `plot_regression_1d()`.

In [ ]:
# Omega = ...
# YOUR CODE HERE
raise NotImplementedError()
theta = weighted_least_squares(X, Omega, y)
plot_regression_1d(X, y, theta, x_min=0, x_max=6)

In [ ]:
assert len(theta) == 2

# Nadaraya-Watson Kernel Regression (Bonus)

Like k-Nearest Neighbors, *Nadaraya-Watson Kernel Regression* is a **non-parametric** model, i.e. decisions are made according to known examples from the *learning set* $\mathcal{D} = \{(y^{(i)}, \boldsymbol{x^{(i)}})\}_{1 \leq i \leq n}$ of $n$ examples and considering a kind of proximity relationship.

With k-Nearest Neighbors, decisions are based only on the closest neighbors and other examples are simply ignored. If you remember correctly from the end of Lab 03, we used `sklearn`'s implementation of k-Nearest Neighbors and its `weights="distance"` option to use the label of the nearest neighbors **proportionally to their distance from the point to predict**.
Contrary to standard k-NN but in the same fashion as the aforementioned option, with Kernel Regression all examples $(\boldsymbol{x}^{(i)}, y^{(i)})$ from $\mathcal{D}$ are used to predict the label $y$ of any new point $\boldsymbol{x}$, but their respective contribution in this prediction is weighted using a *kernel function* $K(\boldsymbol{x}^{(i)}, \boldsymbol{x})$. 

$$
\hat{y}
= f(\boldsymbol{x})
= \frac{\sum^{n}_{i=1} K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) ~ y^{(i)}}{\sum^{n}_{j=1} K(\boldsymbol{x}^{(j)}, \boldsymbol{x})}
= \sum^{n}_{i=1} y^{(i)} \omega^{(i)}
$$

with $\sum^{n}_{i=1} \omega^{(i)} = 1$

Recall about the notation used here:
- $\boldsymbol{x}^{(i)}$ is the feature (input) vector of the $i^{\text{th}}$ example in $\mathcal{D}$ (and $y^{(i)}$ is its label). Beware: $\boldsymbol{x}^{(i)}$ is not the $i^{\text{th}}$ power of $\boldsymbol{x}$ (we will write $\boldsymbol{x}^{(i)2}$ for the square of $\boldsymbol{x}^{(i)}$)!
- $x_i$ is the value of $\boldsymbol{x}$ on the $i^{\text{th}}$ dimension

Implement the Gaussian kernel $K$ in the following `gaussian_kernel()` Python function.

$$
K(\boldsymbol{u}, \boldsymbol{v})
= \exp\left(\frac{-||\boldsymbol{u} - \boldsymbol{v}||^2_2}{2 \sigma^2} \right)
$$

where $\sigma$ is a parameter equal to $1$ by default.

You can assume $u$ and $v$ to be simple scalars to simplify this Python implementation (i.e. restrict yourself to regression problem with 1 dimension inputs $x \in \mathbb{R}$).

Recall: $e^x$ is written `math.exp(x)` in Python.

In [ ]:
def gaussian_kernel(u, v, sigma = 1.):
    # return expression above
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert gaussian_kernel(0,0) == 1
assert gaussian_kernel(1,0) == math.exp(-1/2)

Implement the Nadaraya-Watson kernel regression in the following `kernel_regression()` Python function.

$$
\text{kernel\_regression}(\boldsymbol{x}, \mathcal{D})
= \frac{\sum^{n}_{i=1} K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) y^{(i)}}{\sum^{n}_{j=1} K(\boldsymbol{x}^{(j)}, \boldsymbol{x})}
= \hat{y}
$$

You can assume that $x$ is a scalar to simplify the Python implementation.
We assume `dataset` contains examples $\mathcal{D} = \{(\boldsymbol{x^{(i)}}, y^{(i)})\}_{1 \leq i \leq n}$ in a Pandas DataFrame having:
- one row per example
- a column "x" containing the examples' features (only one dimension here)
- a column "y" containing the examples' labels

**Hint**: you can use the following `for` loop to compute $\sum K(\boldsymbol{x}^{(i)}, \boldsymbol{x}) ~ y^{(i)}$: `for xi, yi in zip(df.x, df.y)`.

In [ ]:
def kernel_regression(x, dataset):
    # Hint: calculate numerator and denominator separately with list comprehensions
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
assert kernel_regression(1, pd.DataFrame({"x": [0], "y": [0]})) == 0

We have the following dataset:

In [ ]:
dataset = pd.DataFrame([[2., 0.],
                        [5., 2.],
                        [7., 1.],
                        [10., 2.],
                        [14., 4.],
                        [16., 3.],
                        [17., 0.]], columns=['x', 'y'])
dataset

Check your `kernel_regression()` function with the following code:

In [ ]:
x_pred = np.linspace(0., 20., 200)
y_pred = [kernel_regression(x, dataset) for x in x_pred]

ax = dataset.plot.scatter(x='x', y='y', label="Dataset", figsize=(12,8))
ax.plot(x_pred, y_pred, "-r", label="Kernel regression")
plt.legend();